In [1]:
%matplotlib widget

import tensorflow as tf
import os
import sys
import cv2
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import random

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from numpy.linalg import norm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [2]:
DATASET_PATH = "/Users/jeremy/Google Drive/datasets/fashion-dataset/"
print(os.listdir(DATASET_PATH))

['embeddings.tsv', '.DS_Store', 'images.csv', 'images', 'styles', 'styles.csv', '.ipynb_checkpoints', 'embeddings.csv', 'resnet50-embeddings.pkl']


In [3]:
df = pd.read_csv(DATASET_PATH + "styles.csv", nrows=5000, error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.reset_index(drop=True)
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,30805.jpg
7,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012,Casual,Jealous 21 Women Purple Shirt,26960.jpg
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012,Casual,Puma Men Pack of 3 Socks,29114.jpg
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016,Casual,Skagen Men Black Watch,30039.jpg


In [4]:
def get_img_path(img):
  return DATASET_PATH + "images/" + img

In [5]:
df_embs = pickle.load(open(DATASET_PATH + "resnet50-embeddings.pkl", "rb"))

In [6]:
# Filter only embeddings with master category 'Apparel' and save indices to filter for them later
df_filtered = df.loc[df.masterCategory == 'Apparel']
df_filtered_idx = df_filtered.index.values.tolist()
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.masterCategory.unique(), df_filtered.shape, len(df_filtered_idx)

(array(['Apparel'], dtype=object), (2297, 11), 2297)

In [7]:
embs_filtered = []
for i, row in df_embs.iterrows():
    if i in  df_filtered_idx:
        embs_filtered.append(row)
df_embs_filtered = pd.DataFrame(embs_filtered)
df_embs_filtered = df_embs_filtered.reset_index(drop=True)
df_embs_filtered

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5.661316,2.366165,0.000000,3.255730,0.845208,2.673841,8.780870,6.280786,0.787188,1.587657,...,2.463610,0.000000,1.825994,20.485268,3.902863,0.000000,8.225969,1.313863,0.000000,10.820526
1,5.790417,9.593648,0.000000,9.431670,0.287235,0.259945,17.028748,3.037453,0.136341,0.000000,...,1.670570,0.013891,10.338681,4.784722,0.189140,0.000000,7.739144,0.372693,0.000000,8.903802
2,5.882203,1.958606,0.000000,12.519882,0.000000,0.000000,9.383594,3.781614,0.000000,1.490617,...,17.825478,0.000000,3.592507,2.042305,0.000000,0.000000,6.656779,1.457314,1.694204,1.403327
3,0.245968,17.383430,0.493824,3.374468,2.567368,0.000000,6.482537,5.862639,0.000000,3.164866,...,4.464024,0.447075,1.448143,12.884138,0.000000,0.397501,5.028075,1.939697,0.000000,11.485400
4,0.028546,18.567099,0.000000,0.757250,1.504949,0.000000,1.982024,2.963777,1.072765,4.539300,...,3.247082,4.273134,3.088643,16.997053,0.000000,0.000000,2.258095,1.595798,0.000000,7.674047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,3.134137,12.337814,3.281830,0.000000,6.123401,6.273666,3.345068,4.230134,2.082114,2.691854,...,5.392367,2.139013,7.540308,5.394151,6.070920,0.000000,0.000000,0.000000,0.000000,10.168349
2293,2.800386,19.866081,3.215006,2.487933,0.147357,0.000000,7.958175,5.669247,0.706661,4.877097,...,10.292748,0.000000,0.738173,16.109261,0.383140,0.000000,4.432935,2.898650,0.416252,12.830476
2294,4.801172,16.536064,2.132150,2.630906,1.190370,0.243107,1.454644,3.500423,0.627453,5.221931,...,0.000000,1.886256,3.294083,17.186272,0.000000,2.014549,1.663332,2.118907,0.000000,6.815829
2295,3.184294,14.538852,6.306130,0.000000,3.361131,6.701039,0.729837,0.295319,0.252367,3.175583,...,0.000000,1.471544,1.482233,22.654318,2.425646,12.427614,8.393599,0.000000,9.616654,3.375090


In [8]:
# Perform PCA over the embeddings to reduce dimensionality
num_feature_dimensions = 200  # Set the number of embedding dimensions
pca = PCA(n_components = num_feature_dimensions)
embs_compressed = pca.fit_transform(df_embs_filtered)
df_embs_filtered_compressed = pd.DataFrame(embs_compressed)
df_embs_filtered_compressed

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-64.281075,-0.382173,10.129713,-22.772631,-38.308014,10.125624,36.410553,-46.646561,-34.147324,8.802277,...,1.060581,5.643126,-3.744165,0.495120,0.968238,-0.283647,4.389657,0.886172,3.537117,2.184576
1,-50.859024,-78.791672,-48.603207,-5.542593,57.489983,-30.834576,-2.337898,-15.355884,-7.670235,-7.935523,...,-3.600985,4.210120,8.553507,-0.847202,-8.451081,2.876735,5.075893,-3.321136,-0.321802,-2.248401
2,-21.894356,-69.084930,-56.701588,-13.024057,48.614300,-35.150684,6.533153,12.986396,1.046110,-23.648092,...,-0.397177,-4.171687,-0.207550,-4.578256,-4.193841,2.557703,0.056334,-5.436918,-3.783812,2.688401
3,-52.814465,-30.695795,24.538797,18.532717,-15.458954,-4.578990,-20.258865,5.052794,-18.399313,14.211680,...,-4.220189,3.145572,-4.264816,-0.480453,-6.051493,1.760078,5.161418,2.619761,-2.452816,-0.218968
4,-26.194763,-12.308652,56.018497,28.274645,4.040713,0.309499,20.196915,11.745950,-15.118042,0.662380,...,-0.578417,0.215504,2.420460,-0.436733,-0.328829,2.788034,1.808488,0.883710,0.061927,-5.266214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,99.575584,-30.713972,-20.068419,-25.830290,-22.532362,51.434589,-40.078949,-24.682877,-38.467953,53.920860,...,0.559850,-0.631072,-2.461216,2.927379,-3.999831,-0.316094,1.079288,-4.650527,-2.159378,0.147086
2293,-26.236422,-12.102029,52.941223,21.009766,-2.619553,-23.604778,3.913764,4.513469,-3.499641,-43.001499,...,4.464515,4.689528,-0.918898,6.069413,-5.554238,-1.010250,2.714449,-5.268404,0.454977,0.248365
2294,-29.318724,36.647499,31.197504,14.335003,26.817417,-5.633993,-21.891531,-2.816190,-21.250416,2.923762,...,1.392772,5.748481,-4.676129,-3.211372,1.707263,6.445720,1.951647,1.729430,-3.171934,-3.996809
2295,54.131577,63.903709,-8.761942,-63.546173,17.439749,9.300364,16.524590,42.935291,53.400822,43.314064,...,2.657250,1.502560,9.862597,-3.212071,4.481094,-5.022458,-1.849408,4.443483,8.119142,-3.160293


In [9]:
# Map season values to either Spring or Winter
for i, row in df_filtered.iterrows():
    if df_filtered.at[i, 'season'] == "Spring":
        df_filtered.at[i, 'season'] = "Summer"
    elif df_filtered.at[i, 'season'] == "Fall":
        df_filtered.at[i, 'season'] = "Winter"
df_filtered.season.unique()

array(['Winter', 'Summer'], dtype=object)

In [10]:
# Application of SVM to create the axis for a given feature
X = df_embs_filtered_compressed
y = df_filtered["season"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
svm_clf = LinearSVC(C=1, max_iter=100000)
svm_clf.fit(X_scaled, y)

LinearSVC(C=1, max_iter=100000)

In [11]:
def get_nearest_neighbour(user_position, df, embs):
   nearest_neighbour = None
   nearest_neighbour_pos = None
   smallest_dist = sys.maxsize
   for row in df.itertuples():
      embedding_position = embs.iloc[row.Index].to_numpy()
      dist = norm(user_position - embedding_position)
      if dist < smallest_dist:
         smallest_dist = dist
         nearest_neighbour = row.id
         nearest_neighbour_pos = embedding_position
   return nearest_neighbour, nearest_neighbour_pos

In [12]:
def annotate_nearest_neighbour(nearest_neighbour, ax, df):
    arr_img = plt.imread(get_img_path(df.loc[df['id'] == nearest_neighbour].image.values[0]))
    plt.imshow(arr_img)

In [14]:
def compute_navigation_axis(emb, w, range):
    nav_axis = []
    for i, feature in enumerate(emb):
        feature_val = emb[feature].values[0]
        if w[i] > 0:
            pos_range = np.arange(start=feature_val, stop=feature_val + range, step=w[i])
            neg_range = np.arange(start=feature_val, stop=feature_val - range, step=-w[i])
            nav_axis.append(np.sort(np.concatenate((pos_range, neg_range))))
        else:
            pos_range = np.arange(start=feature_val, stop=feature_val + range, step=-w[i])
            neg_range = np.arange(start=feature_val, stop=feature_val - range, step=w[i])
            nav_axis.append(np.sort(np.concatenate((pos_range, neg_range))))
    return nav_axis

In [37]:
def initialize_user_position(axis, idx):
    user_position = []
    for dim in axis:
        user_position.append(dim[idx])
    return user_position

In [40]:
def update_user_position(change):
    """Update the user position after the slider value has changed"""
    new_x, new_y, new_z = get_updated_user_pos(change.new, nav_axis[0], nav_axis[1], nav_axis[2])
    new_user_pos = np.array([new_x, new_y, new_z])
    user_position = new_user_pos
    user_marker[0].set_data(new_x, new_y)
    user_marker[0].set_3d_properties(new_z)
    nearest_neighbour, nearest_neighbour_pos = get_nearest_neighbour(new_user_pos, df_filtered)
    annotate_nearest_neighbour(nearest_neighbour, ax2, df_filtered)
    fig.suptitle('Nearest Embedding: {} with season: {}, current user position: {}'.format(nearest_neighbour, df_filtered.loc[df_filtered['id'] == nearest_neighbour].season.values[0], user_position))
    fig.canvas.draw()
    fig.canvas.flush_events()

In [39]:
from ipywidgets import AppLayout, FloatSlider

# Create matplotlib figure for displaying fashion items
fig = plt.figure(figsize=(7, 7))

# Create orthogonal navigation axis
w = svm_clf.coef_[0]
rand_emb = df_embs_filtered_compressed.sample()
nav_axis = compute_navigation_axis(rand_emb, w, 100)

starting_idx = np.where(np.isclose(nav_axis[0], min(nav_axis[0], key=lambda x:abs(x-rand_emb[0].values[0]))))[0][0]
user_position = initialize_user_position(nav_axis, starting_idx)

# Create Slider to navigate in embedding space
slider = FloatSlider(
    orientation="horizontal",
    description="x-Position:",
    value=user_position[0],
    min=min(nav_axis[0][:1200]),
    max=max(nav_axis[0][:1200])
)
slider.layout.margin = '0px 20% 0px 20%'
slider.layout.width = '30%'

slider.observe(update_user_position, names='value')

display(slider)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'update_user_position' is not defined